In [1]:
# Dependencies
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy import *
from config import username, password

In [42]:
dfPop2000 = pd.DataFrame
csvFile = 'Resources/population_by_zip_2000.csv'
dfPop2000 = pd.read_csv(csvFile,delimiter=',',encoding='utf-8',low_memory=False,dtype=str)
dfPop2010 = pd.DataFrame
csvFile = 'Resources/population_by_zip_2010.csv'
dfPop2010 = pd.read_csv(csvFile,delimiter=',',encoding='utf-8',low_memory=False,dtype=str)
dfBars = pd.DataFrame
csvFile = 'Resources/8260_1.csv'

dfBars = pd.read_csv(csvFile,delimiter=',',encoding='utf-8',low_memory=False,dtype=str)
dfBars.dropna(subset=['postalCode'],inplace=True) # Drop rows with no zip code listed
dfBars['postalCode'] = dfBars['postalCode'].str.zfill(5) # Prepend '0' if zip code less than 5 digits long
dfBars['postalCode'] = dfBars['postalCode'].str.slice(start=0,stop=5) # Keep only first 5 digits of zip code
bars_columns = [u'postalCode',u'name']
bars_clean = dfBars[bars_columns].copy()
dfBars=bars_clean.rename(columns={'postalCode':'zipcode'})


ZipBars = dfBars.groupby(['zipcode']).count()
ZipBars.sort_values('name',ascending=False,inplace=True)
ZipBars.rename(columns={'name':'bar_count'},inplace=True)

dfPop2010.dropna(subset=['zipcode'],inplace=True)
dfPop2010['zipcode'] = dfPop2010['zipcode'].str.zfill(5)
dfPop2010['zipcode'] = dfPop2010['zipcode'].str.slice(start=0,stop=5)

dfPop2000.dropna(subset=['zipcode'],inplace=True)
dfPop2000['zipcode'] = dfPop2000['zipcode'].str.zfill(5)
dfPop2000['zipcode'] = dfPop2000['zipcode'].str.slice(start=0,stop=5)


In [43]:
# Create connection to Bars_db
rds_connection_string = username + ':' + password + '@127.0.0.1/Bars_db'
engine = create_engine(f'mysql://{rds_connection_string}')

In [50]:
ZipBars.to_sql(name='bars_per_zipcode', con=engine, if_exists='replace', index=True, index_label='zipcode',dtype={'zipcode':String(5)})
pd.read_sql_query('select * from bars_per_zipcode', con=engine).head()
dfBars.head()

,zipcode,name,id
1,53212,The Waterfront Cafe,1
2,93612,559 Local Brewing,2
3,53515,Shack The,3
4,47130,Flat12 Bierworks,4
5,98072,Redhook Ale Brewery Inc,5


In [58]:
id = dfBars.index.tolist()
dfBars['id'] = str(id)
dfBars['id'] = dfBars[u'id']
dfBars['zipcode'] = dfBars[u'zipcode']
dfBars['name'] = dfBars[u'name']
dfBars.index

Int64Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,
               10,
            ...
            17813, 17814, 17815, 17816, 17817, 17818, 17819, 17820, 17821,
            17822],
           dtype='int64', length=17198)

In [66]:
dfBars.to_sql(name='bar_names', con=engine, if_exists='replace',index=False,dtype={'id':Integer,'zipcode':String(5),'name':String(50)})
engine.table_names()

OperationalError: (MySQLdb._exceptions.OperationalError) (1366, "Incorrect integer value: '[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, ' for column 'id' at row 1")
[SQL: INSERT INTO bar_names (zipcode, name, id) VALUES (%s, %s, %s)]
[parameters: (('53212', 'The Waterfront Cafe', '[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 4 ... (109405 characters truncated) ... 9, 17801, 17802, 17804, 17805, 17806, 17807, 17808, 17809, 17810, 17811, 17812, 17813, 17814, 17815, 17816, 17817, 17818, 17819, 17820, 17821, 17822]'), ('93612', '559 Local Brewing', '[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 4 ... (109405 characters truncated) ... 9, 17801, 17802, 17804, 17805, 17806, 17807, 17808, 17809, 17810, 17811, 1

In [17]:


engine.table_names()

ValueError: duplicate name in index/columns: cannot insert zipcode, already exists